In [6]:
from experiments.model_training.train_qwen import get_tokenizer, QWEN25_TEMPLATE_PATH, check_chat_template_supports_assistant_only_loss

model_name = "Qwen/Qwen2.5-3B-instruct"
tokenizer = get_tokenizer(model_name, chat_template_path=QWEN25_TEMPLATE_PATH)

print(check_chat_template_supports_assistant_only_loss(tokenizer.chat_template))

Loading tokenizer for Qwen/Qwen2.5-3B-instruct
Tokenizer loaded in 0.68 seconds
Tokenizer chat_template loaded from /home/ubuntu/victor-north-tx/tau2-bench-private/src/experiments/model_training/qwen2.5_prompt_template.jinja
Tokenizer has a chat template.
Tokenizer eos_token = <|im_end|> (id: 151645)
Tokenizer pad_token = <|endoftext|> (id: 151643)
Tokenizer model_max_length: 131072
Tokenizer chat_template supports assistant only loss: True
True


In [15]:
from experiments.model_training.train_qwen import DEFAULT_TRAIN_DATASET_PATH, DEFAULT_TEST_DATASET_PATH, DEFAULT_MAX_TRAIN_DATAPOINTS, DEFAULT_MAX_TEST_DATAPOINTS
from experiments.model_training.prepare_dataset import load_as_hf_dataset

train_dataset = load_as_hf_dataset(DEFAULT_TRAIN_DATASET_PATH, max_datapoints=DEFAULT_MAX_TRAIN_DATAPOINTS)
test_dataset = load_as_hf_dataset(DEFAULT_TEST_DATASET_PATH, max_datapoints=DEFAULT_MAX_TEST_DATAPOINTS)


In [8]:
train_dataset[0]["messages"][1]

{'content': 'Hi! How can I help you today?',
 'role': 'assistant',
 'tool_call_id': None,
 'tool_calls': None}

In [11]:
res = tokenizer.apply_chat_template(train_dataset[0]["messages"], tools = train_dataset[0]["tools"], return_dict=True, return_assistant_tokens_mask=True)


In [14]:
print(tokenizer.apply_chat_template(train_dataset[0]["messages"], tokenize=False))

<|im_start|>system
<instructions>
You are a customer service agent that helps the user according to the <policy> provided below.
During each turn you can either:
- Send a message to the user.
- Make a tool call.
IMPORTANT: You cannot do both at the same time!!
If you send text content while making a tool call, the agent will raise an error.
Text content will be sent to the user. Do not use this field for your own reasoning.

Try to be helpful and always follow the policy. Always make sure you generate valid JSON only.
</instructions>
<policy>
# Airline Agent Policy

The current time is 2024-05-15 15:00:00 EST.

As an airline agent, you can help users **book**, **modify**, or **cancel** flight reservations. You also handle **refunds and compensation**.

Before taking any actions that update the booking database (booking, modifying flights, editing baggage, changing cabin class, or updating passenger information), you must list the action details and obtain explicit user confirmation (ye

In [20]:
assistant_token_ids = [token_id for i, token_id in enumerate(res["input_ids"]) if res["assistant_masks"][i]]
print(" ".join([tokenizer.decode(token_id) for token_id in assistant_token_ids]))





 Hi !  How  can  I  help  you  today ? <|im_end|> 

 

 According  to  the  policy ,  travel  insurance  cannot  be  canceled  or  removed  after  the  initial  booking .  Therefore ,  I  am  unable  to  assist  with  cancel ing  the  insurance  separately  from  the  flight .

 If  you  have  any  other  requests  or  need  assistance  with  something  else ,  please  let  me  know . <|im_end|> 

 

 <tool_call> 
 {" name ":  " transfer _to _human _agents ",  " arguments ":  "{\" summary \":  \" User  is  unhappy  with  the  policy  that  travel  insurance  cannot  be  canceled  or  refunded  separately  from  the  flight .  User  wants  to  keep  the  flight  but  get  a  refund  for  the  insurance  and  is  requesting  escalation  or  exception .\" }" }
 </tool_call> <|im_end|> 

 

 YOU  ARE  BE ING  TRANS FER RED  TO  A  H UMAN  AG ENT .  PLEASE  HOLD  ON . <|im_end|> 




In [20]:
res = tokenizer.apply_chat_template(train_dataset[0]["messages"], tokenize=True, return_dict=True, add_generation_prompt=False, return_assistant_tokens_mask=True)
print(res)

{'input_ids': [151644, 8948, 198, 27, 62295, 397, 2610, 525, 264, 6002, 2473, 8315, 429, 8609, 279, 1196, 4092, 311, 279, 366, 34790, 29, 3897, 3685, 624, 16014, 1817, 2484, 498, 646, 2987, 510, 12, 11000, 264, 1943, 311, 279, 1196, 624, 12, 7405, 264, 5392, 1618, 624, 98743, 25, 1446, 4157, 653, 2176, 518, 279, 1852, 882, 50347, 2679, 498, 3624, 1467, 2213, 1393, 3259, 264, 5392, 1618, 11, 279, 8315, 686, 4828, 458, 1465, 624, 1178, 2213, 686, 387, 3208, 311, 279, 1196, 13, 3155, 537, 990, 419, 2070, 369, 697, 1828, 32711, 382, 21453, 311, 387, 10950, 323, 2677, 1795, 279, 4842, 13, 23240, 1281, 2704, 498, 6923, 2697, 4718, 1172, 624, 522, 62295, 397, 7926, 8018, 397, 2, 6553, 1056, 20713, 10974, 271, 785, 1482, 882, 374, 220, 17, 15, 17, 19, 12, 15, 20, 12, 16, 20, 220, 16, 20, 25, 15, 15, 25, 15, 15, 25877, 382, 2121, 458, 32475, 8315, 11, 498, 646, 1492, 3847, 3070, 2190, 97219, 3070, 34053, 97219, 476, 3070, 18515, 334, 10971, 40062, 13, 1446, 1083, 3705, 3070, 1097, 42950, 323, 1

In [ ]:
# QWEN25_TEMPLATE = "{%- if tools %}\n    {{- '<|im_start|>system\\n' }}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- messages[0]['content'] }}\n    {%- else %}\n        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}\n    {%- endif %}\n    {{- \"\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\" }}\n    {%- for tool in tools %}\n        {{- \"\\n\" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- \"\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\\"name\\\": <function-name>, \\\"arguments\\\": <args-json-object>}\\n</tool_call><|im_end|>\\n\" }}\n{%- else %}\n    {%- if messages[0]['role'] == 'system' %}\n        {{- '<|im_start|>system\\n' + messages[0]['content'] + '<|im_end|>\\n' }}\n    {%- else %}\n        {{- '<|im_start|>system\\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\\n' }}\n    {%- endif %}\n{%- endif %}\n{%- for message in messages %}\n    {%- if (message.role == \"user\") or (message.role == \"system\" and not loop.first) %}\n        {{- '<|im_start|>' + message.role + '\\n' + message.content + '<|im_end|>' + '\\n' }}\n    {%- elif message.role == \"assistant\" %}\n       {% generation %}\n       {{- '<|im_start|>' + message.role }}\n        {%- if message.content %}\n            {{- '\\n' + message.content }}\n        {%- endif %}\n        {%- for tool_call in message.tool_calls %}\n            {%- if tool_call.function is defined %}\n                {%- set tool_call = tool_call.function %}\n            {%- endif %}\n            {{- '\\n<tool_call>\\n{\"name\": \"' }}\n            {{- tool_call.name }}\n            {{- '\", \"arguments\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- '}\\n</tool_call>' }}\n        {%- endfor %}\n        {{- '<|im_end|>\\n' }}\n       {% endgeneration %}\n    {%- elif message.role == \"tool\" %}\n        {%- if (loop.index0 == 0) or (messages[loop.index0 - 1].role != \"tool\") %}\n            {{- '<|im_start|>user' }}\n        {%- endif %}\n        {{- '\\n<tool_response>\\n' }}\n        {{- message.content }}\n        {{- '\\n</tool_response>' }}\n        {%- if loop.last or (messages[loop.index0 + 1].role != \"tool\") %}\n            {{- '<|im_end|>\\n' }}\n        {%- endif %}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|im_start|>assistant\\n' }}\n{%- endif %}\n"


{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

In [9]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."}
]

res = tokenizer.apply_chat_template(messages, tokenize=True, return_dict=True, add_generation_prompt=False, return_assistant_tokens_mask=True)
print(res)
[tokenizer.decode(token_id) for i, token_id in enumerate(res['input_ids']) if res['assistant_masks'][i]]

{'input_ids': [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 3838, 374, 279, 6722, 315, 9625, 30, 151645, 198, 151644, 77091, 198, 785, 6722, 315, 9625, 374, 12095, 13, 151645, 271], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'assistant_masks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


['<|im_start|>',
 'assistant',
 '\n',
 'The',
 ' capital',
 ' of',
 ' France',
 ' is',
 ' Paris',
 '.',
 '<|im_end|>',
 '\n\n']